In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baqir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\baqir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\baqir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
import pandas as pd     
import re   
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
import spacy
spacy.cli.download("en_core_web_sm")        
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
pd.set_option('display.max_colwidth',None)  
try:    
    df = pd.read_csv(r"D:\dataset_of_project\imdb _dataset\IMDB Dataset.csv")   
except FileNotFoundError:   
    print("Error : DATASET NOT FOUND")      

print("First two rows of dataset:")    
display(df.head(2))

First two rows of dataset:


,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive


In [8]:
print("dataset info")   
df.info(    
    
)

dataset info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
print("sentiment distribution") 
print(df['sentiment'].value_counts())

sentiment distribution
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [10]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [11]:
def preprocess_text(text):
    """
    Cleans and preprocesses a single text string.
    1. Removes HTML tags
    2. Lowercases text
    3. Removes punctuation and numbers
    4. Tokenizes text
    5. Removes stop words
    6. Lemmatizes words
    """
    text = re.sub(r'<.*?>', '', text)
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization is often better than stemming
    text = ' '.join(tokens)
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    
    return ' '.join(lemmatized_tokens)

In [12]:
print("preproccesing all data take time")   
df = df.sample(500).copy()  
df['cleaned_review'] = df['review'].apply(preprocess_text)  
print("/n compare original vs cleaned") 
display(df[['review','cleaned_review']].head())

preproccesing all data take time
/n compare original vs cleaned


,review,cleaned_review
10328,"Kudos to the writers of this film for creating a supremely engaging drama. The curious character development is indicative of a nuanced and well schooled writing team. The audience member cannot but help but to feel that (s)he must make wrenching emotional decisions pitting the cerebral against the libidinal. The viewer has an opportunity to develop the character herself, though her predictions are rarely validated.<br /><br />Credit is also due to the filmmakers for breathing life into the setting. The wood-shop is transformed into a unique persona as the film unfolds, with its own traits, faults, a variety of highly charged relationships, and of course a fate inexorably tied to that of the other principals.<br /><br />Make sure to catch this one at your local art-house.",kudos writer film create supremely engaging drama curious character development indicative nuance well school write team audience member help feel must make wrench emotional decision pit cerebral libidinal viewer opportunity develop character though prediction rarely validatedcredit also due filmmaker breathe life set woodshop transform unique persona film unfold trait fault variety highly charge relationship course fate inexorably tie principalsmake sure catch one local arthouse
948,i think it is great one of my favourite films as a kid and who said there songs were unforgettable they were mint i can still remember them now WORD FOR WORD the film remains a favourite with my family and my younger cousins are now addicted to it too they even know the songs this film is great and a enjoyable film for kids it has a moral lesson so don't say its not good because it shows how lying gets you nowhere ill leave with a parting comment: this film is amazing love me xxx P.s i would like the soundtrack but i cant and yes the animation is good the jokes are humorous and the action never stops.This film will go down in children's film history and in my opinion one of the only remaining safe films to show children.,think great one favourite film kid say song unforgettable mint still remember word word film remain favourite family young cousin addict even know song film great enjoyable film kid moral lesson do not say good show lie get nowhere ill leave part comment film amazing love xxx ps would like soundtrack can not yes animation good joke humorous action never stopsthis film go children film history opinion one remain safe film show child
10262,"I found this movie to be suspenseful almost from the get-go. When Miss Stanwyck starts her narration it's only a few minutes until you realize that trouble is coming. The deserted area, the lock on the deserted gas station door, everything sets you up to wait for it...here it comes. At first you think it will be about the little boy, but all too soon you start holding your breath watching the tide coming in. I found this movie to be really stressful, even though I had watched it before and was prepared for the denouement. Now a movie that can keep you in suspense even when you have seen it before deserves some sort of special rating, maybe a white knuckles award?",find movie suspenseful almost getgo miss stanwyck start narration minute realize trouble come deserted area lock deserted gas station door everything set wait ithere come first think little boy soon start hold breath watch tide come find movie really stressful even though watch prepare denouement movie keep suspense even see deserve sort special rating maybe white knuckle award
36690,"When you get ahead of a film, you know you are wasting your time watching the movie in question. That is exactly how I felt while I was watching ""Darkness."" I could see anticipate every twist and turn easily. It is a combination of ""The Amityville Horror"" and a myriad of films that deal with the occult. It is so silly that I almost found endearing its naiveness. I cannot say I was wholly bored but I found the film way too derivative for my taste. Directo

In [13]:
from sklearn.model_selection import train_test_split

# Map sentiment labels to 0 and 1
df['sentiment_numeric'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Define our features (X) and target (y)
X = df['cleaned_review']
y = df['sentiment_numeric']

# Split the data into training and testing sets (80% train, 20% test)
# stratify=y ensures the train/test split has the same proportion of sentiments as the original dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

Training set size: 400
Testing set size: 100


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
# max_features=5000 means we only use the 5000 most common words, which is efficient
tfidf = TfidfVectorizer(max_features=500)

# Fit the vectorizer on the training data and transform it
X_train_tfidf = tfidf.fit_transform(X_train)

# ONLY transform the test data using the already-fitted vectorizer
X_test_tfidf = tfidf.transform(X_test)

print(f"Shape of the training TF-IDF matrix: {X_train_tfidf.shape}")
print(f"Shape of the testing TF-IDF matrix: {X_test_tfidf.shape}")

Shape of the training TF-IDF matrix: (400, 500)
Shape of the testing TF-IDF matrix: (100, 500)


In [15]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the model
# solver='liblinear' is a good choice for this type of problem
model = LogisticRegression(solver='liblinear')
model.fit(X_train_tfidf, y_train)

print("Model training complete!")

Model training complete!


In [18]:
def predict_sentiment(text):
    """
    Takes a raw text string and predicts its sentiment using the trained model.
    """
    # 1. Preprocess the text using the same function from Cell 2
    cleaned_text = preprocess_text(text)
    
    # 2. Vectorize the text using the SAME fitted TF-IDF vectorizer from Cell 5
    vectorized_text = tfidf.transform([cleaned_text])
    
    # 3. Predict using the trained model from Cell 6
    prediction = model.predict(vectorized_text)
    
    # 4. Return the human-readable result
    return 'Positive' if prediction[0] == 1 else 'Negative'

# --- Test Cases ---
review_1 = "This movie was absolutely bad! The acting was normal and the plot was not good."
review_2 = "I was so bored throughout the entire film. It was a complete waste of time and money."
review_3 = "The film was good, great but not terrible either. Some parts were good."

print(f"Review: '{review_1}'\nPredicted Sentiment: {predict_sentiment(review_1)}\n")
print(f"Review: '{review_2}'\nPredicted Sentiment: {predict_sentiment(review_2)}\n")
print(f"Review: '{review_3}'\nPredicted Sentiment: {predict_sentiment(review_3)}\n")

Review: 'This movie was absolutely bad! The acting was normal and the plot was not good.'
Predicted Sentiment: Negative

Review: 'I was so bored throughout the entire film. It was a complete waste of time and money.'
Predicted Sentiment: Negative

Review: 'The film was good, great but not terrible either. Some parts were good.'
Predicted Sentiment: Positive

